# Generate segmentation masks from predictions

7.02.2022

Uso questo script per trasformare i raw outputs della unet in un unico video che contiene la segmentazione dell'input (cioè con valori compresi in {0,1,2,3}).

L'idea è poi di automatizzare la generazione di questi output "binari" per tutti i video di un certo modello (training) in uno script .py e di eliminare questo notebook.

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import glob
import os
import imageio

from scipy.ndimage.morphology import binary_dilation, binary_erosion
from sklearn.metrics import jaccard_score, f1_score

import pandas as pd
import matplotlib.pyplot as plt

import unet
from metrics_tools import (correspondences_precision_recall, 
                           Metrics, 
                           reduce_metrics, 
                           empty_marginal_frames,
                           process_spark_prediction,
                           process_puff_prediction,
                           process_wave_prediction,
                           jaccard_score_exclusion_zone,
                           write_videos_on_disk,
                           get_sparks_locations_from_mask,
                           compute_prec_rec,
                           reduce_metrics_thresholds
                          )
from dataset_tools import final_mask
#from generate_unet_annotations import final_mask # TODO: uncomment and fix generate_unet_annotations with main function

In [3]:
BASEDIR = os.path.abspath('')
BASEDIR

'C:\\Users\\dotti\\sparks_project\\sparks'

### Select predictions to load

In [4]:
training_names = [#"temporal_reduction_ubelix",
                  #"256_long_chunks_ubelix",
                  #"focal_loss_ubelix",
                  #"pretrained_only_sparks_ubelix",
                  #"only_sparks_ubelix",
                  "focal_loss_gamma_5_ubelix"
                  ]

epoch = 100000

# if a training has a different load epoch
epoch_2 = 200000 
epoch_2_training_name = "pretrained_only_sparks_ubelix"

# if loading a training that uses temporal reduction, predictions have to be adapted
temp_red_training_name = "temporal_reduction_ubelix"

### Configure input/output folder

In [5]:
validation_folder = "trainings_validation"

### Load annotations
open and process annotations 

In [6]:
ys_all_trainings = {} # indexed by video number

ys_filenames = sorted(glob.glob(os.path.join(validation_folder, "[0-9]*_video_mask.tif")))

for f in ys_filenames:
    video_id = os.path.split(f)[1][:2]
    ys_all_trainings[video_id] = np.asarray(imageio.volread(f)).astype('int')

In [7]:
ys_all_trainings.keys()

dict_keys(['05', '10', '15', '20', '25', '32', '34', '40', '45'])

### Load predictions

In [24]:
ys = {} # contains annotations for each training: num trainings (dict) x num videos (dict) x video shape
sparks = {} # contains sparks for each training: num trainings x num videos x video shape
puffs = {} # contains puffs for each training: num trainings x num videos x video shape
waves = {} # contains waves for each training: num trainings x num videos x video shape

for training_name in training_names:
    # Import .tif files as numpy array
    load_epoch = epoch_2 if training_name == epoch_2_training_name else epoch
    base_name = os.path.join(validation_folder,training_name+"_"+str(load_epoch)+"_")
    
    if training_name == temp_red_training_name: # need to use annotations from another training
        if training_names[-1] != temp_red_training_name:
            base_name_ys = os.path.join(validation_folder,training_names[-1]+"_"+str(load_epoch)+"_")
            ys_filenames = sorted(glob.glob(base_name_ys+"[0-9][0-9]_video_ys.tif"))
        else:
            print("SELECT TRAINING NAME DIFFERENT FROM temporal_reduction")
            break
    else:    
        ys_filenames = sorted(glob.glob(base_name+"[0-9][0-9]_video_ys.tif"))
        
    sparks_filenames = sorted(glob.glob(base_name+"[0-9][0-9]_video_sparks.tif"))
    puffs_filenames = sorted(glob.glob(base_name+"[0-9][0-9]_video_puffs.tif"))
    waves_filenames = sorted(glob.glob(base_name+"[0-9][0-9]_video_waves.tif"))
        
    training_ys = {}
    training_sparks = {}
    training_puffs = {}
    training_waves = {}
    
    for idx,y,s,p,w in zip(ys_all_trainings.keys(),ys_filenames,sparks_filenames,puffs_filenames,waves_filenames):
        ys_loaded = np.asarray(imageio.volread(y)).astype('int')
        training_ys[idx] = ys_loaded
        
        if training_name == temp_red_training_name: # repeat each frame 4 times
            s_preds = np.asarray(imageio.volread(s))
            p_preds = np.asarray(imageio.volread(p))
            w_preds = np.asarray(imageio.volread(w))
                        
            # repeat predicted frames x4
            s_preds = np.repeat(s_preds,4,0)
            p_preds = np.repeat(p_preds,4,0)
            w_preds = np.repeat(w_preds,4,0)
            
            # if original length %4 != 0, crop preds
            if ys_loaded.shape != s_preds.shape:
                duration = ys_loaded.shape[0]
                s_preds = s_preds[:duration]
                p_preds = p_preds[:duration]
                w_preds = w_preds[:duration]
            
            assert ys_loaded.shape == s_preds.shape
            assert ys_loaded.shape == p_preds.shape
            assert ys_loaded.shape == w_preds.shape
            
            training_sparks[idx] = s_preds
            training_puffs[idx] = p_preds
            training_waves[idx] = w_preds
        else:
            training_sparks[idx] = np.asarray(imageio.volread(s))
            training_puffs[idx] = np.asarray(imageio.volread(p))
            training_waves[idx] = np.asarray(imageio.volread(w))

    ys[training_name] = training_ys
    sparks[training_name] = training_sparks
    puffs[training_name] = training_puffs
    waves[training_name] = training_waves        

### Select sample training and sample video

In [10]:
# all video indices:
print(*list(ys_all_trainings.keys()))

05 10 15 20 25 32 34 40 45


In [11]:
training_name = training_names[0]
video_id = '34'

sample_ys = ys[training_name][video_id]
sample_sparks = sparks[training_name][video_id]
sample_puffs = puffs[training_name][video_id]
sample_waves = waves[training_name][video_id]

### Generate sample binary prediction

In [9]:
# general params
ignore_frames_loss = 6

In [ ]:
# physiological params
PIXEL_SIZE = 0.2 # 1 pixel = 0.2 um x 0.2 um
global MIN_DIST_XY
MIN_DIST_XY = round(1.8 / PIXEL_SIZE) # min distance in space between sparks
TIME_FRAME = 6.8 # 1 frame = 6.8 ms
global MIN_DIST_T
MIN_DIST_T = round(20 / TIME_FRAME) # min distance in time between sparks

In [10]:
# process sparks preds 

# get sparks centres
t_sparks = 0.7
min_radius_sparks = 2
binary_sparks = process_spark_prediction(pred=sample_sparks,
                                         t_detection=t_sparks,
                                         min_dist_xy = MIN_DIST_XY,
                                         min_dist_t = MIN_DIST_T,
                                         min_radius=min_radius_sparks,
                                         return_mask=True,
                                         ignore_frames=ignore_frames_loss
                                        )[1]

NameError: name 'sample_sparks' is not defined

In [15]:
# increase sparks dimension
radius_event = 3
radius_ignore = 1
ignore_index = 4
binary_sparks = final_mask(mask=binary_sparks,
                           radius1=radius_event,
                           radius2=radius_event+radius_ignore,
                           ignore_ind=ignore_index
                          )

In [17]:
binary_sparks.shape, np.unique(binary_sparks)

((904, 64, 512), array([0, 1, 4], dtype=int64))

In [18]:
# process puffs and waves preds to get binary masks
t_puffs = 0.5
t_waves = 0.5
min_radius_puffs = 4
min_radius_waves = 6

binary_puffs = process_puff_prediction(pred=sample_puffs, 
                                       t_detection=t_puffs, 
                                       min_radius=min_radius_puffs, 
                                       ignore_frames=ignore_frames_loss
                                      )
binary_waves = process_wave_prediction(pred=sample_waves,
                                       t_detection=t_waves, 
                                       min_radius=min_radius_waves, 
                                       ignore_frames=ignore_frames_loss
                                      )

In [33]:
# combine all classes into a unique prediction
binary_puffs_waves = 3*binary_puffs + 2*binary_waves
assert np.all(binary_puffs_waves) < 5, "ROI in binary mask true for both puffs and waves"

In [36]:
binary_mask = np.where(np.isin(binary_sparks, [1,4]), binary_sparks, binary_puffs_waves)

In [37]:
imageio.volwrite(os.path.join(validation_folder, "TEST.tif"), np.uint8(binary_mask))

### Generate binary annotations for all training and all videos

In [25]:
# physiological params
PIXEL_SIZE = 0.2 # 1 pixel = 0.2 um x 0.2 um
global MIN_DIST_XY
MIN_DIST_XY = round(1.8 / PIXEL_SIZE) # min distance in space between sparks
TIME_FRAME = 6.8 # 1 frame = 6.8 ms
global MIN_DIST_T
MIN_DIST_T = round(20 / TIME_FRAME) # min distance in time between sparks

In [26]:
# general params
ignore_frames_loss = 6

'''# sparks params
t_sparks = 0.9
min_radius_sparks = 2
radius_event = 3
radius_ignore = 1
ignore_index = 4

# puffs and waves params
t_puffs = 0.5
t_waves = 0.5
min_radius_puffs = 8
min_radius_waves = 12'''

# for focal_loss_gamma_5_ubelix
# sparks params
t_sparks = 0.7
min_radius_sparks = 2
radius_event = 3
radius_ignore = 1
ignore_index = 4

# puffs and waves params
t_puffs = 0.65
t_waves = 0.65
min_radius_puffs = 6
min_radius_waves = 8

In [27]:
from metrics_tools import process_spark_prediction
for training_name in training_names:
    print(f"processing training {training_name}...")
    for video_id in ys_all_trainings.keys():
        print(f"\tprocessing sample {video_id}...")
        sample_ys = ys[training_name][video_id]
        sample_sparks = sparks[training_name][video_id]
        sample_puffs = puffs[training_name][video_id]
        sample_waves = waves[training_name][video_id]
        
        # process sparks preds 
        binary_sparks = process_spark_prediction(pred=sample_sparks,
                                                 t_detection=t_sparks,
                                                 min_dist_xy = MIN_DIST_XY,
                                                 min_dist_t = MIN_DIST_T,
                                                 min_radius=min_radius_sparks,
                                                 return_mask=True,
                                                 ignore_frames=ignore_frames_loss
                                                )[1]
        # increase sparks dimension
        binary_sparks = final_mask(mask=binary_sparks,
                                   radius1=radius_event,
                                   radius2=radius_event+radius_ignore,
                                   ignore_ind=ignore_index
                                  )
        
        # process puffs and waves preds to get binary masks
        binary_puffs = process_puff_prediction(pred=sample_puffs, 
                                               t_detection=t_puffs, 
                                               min_radius=min_radius_puffs, 
                                               ignore_frames=ignore_frames_loss
                                              )
        binary_waves = process_wave_prediction(pred=sample_waves,
                                               t_detection=t_waves, 
                                               min_radius=min_radius_waves, 
                                               ignore_frames=ignore_frames_loss
                                              )
        
        # combine all classes into a unique prediction
        binary_puffs_waves = 3*binary_puffs + 2*binary_waves
        assert np.all(binary_puffs_waves) < 5, "ROI in binary mask true for both puffs and waves"
        binary_mask = np.where(np.isin(binary_sparks, [1,4]), binary_sparks, binary_puffs_waves)
        
        # write binary preds to disk
        load_epoch = epoch_2 if  training_name == epoch_2_training_name else epoch 
        filename = training_name + "_" + str(load_epoch) + "_" + video_id + "_pred_segmentation.tif"
        imageio.volwrite(os.path.join(validation_folder, filename), np.uint8(binary_mask))

processing training focal_loss_gamma_5_ubelix...
	processing sample 05...
ellipsoid shape (5, 11, 11)
smoothed video shape (500, 64, 512)
	processing sample 10...
ellipsoid shape (5, 11, 11)
smoothed video shape (500, 64, 512)
	processing sample 15...
ellipsoid shape (5, 11, 11)
smoothed video shape (500, 64, 512)
	processing sample 20...
ellipsoid shape (5, 11, 11)
smoothed video shape (500, 64, 512)
	processing sample 25...
ellipsoid shape (5, 11, 11)
smoothed video shape (1000, 64, 512)
	processing sample 32...
ellipsoid shape (5, 11, 11)
smoothed video shape (1000, 64, 512)
	processing sample 34...
ellipsoid shape (5, 11, 11)
smoothed video shape (904, 64, 512)
	processing sample 40...
ellipsoid shape (5, 11, 11)
smoothed video shape (1000, 64, 512)
	processing sample 45...
ellipsoid shape (5, 11, 11)
smoothed video shape (1000, 64, 512)


### Generate binary masks where sparks are unprocessed

In [15]:
for training_name in training_names:
    for video_id in ys_all_trainings.keys():
        sample_ys = ys[training_name][video_id]
        sample_sparks = sparks[training_name][video_id]
        sample_puffs = puffs[training_name][video_id]
        sample_waves = waves[training_name][video_id]
        
        # process sparks preds 
        binary_sparks = process_spark_prediction(pred=sample_sparks,
                                                 t_detection=t_sparks,
                                                 min_radius=min_radius_sparks,
                                                 return_clean_pred=True,
                                                 ignore_frames=ignore_frames_loss
                                                ) > t_sparks
    
        # increase sparks dimension
        '''binary_sparks = final_mask(mask=binary_sparks,
                                   radius1=radius_event,
                                   radius2=radius_event+radius_ignore,
                                   ignore_ind=ignore_index
                                  )'''
        
        # process puffs and waves preds to get binary masks
        binary_puffs = process_puff_prediction(pred=sample_puffs, 
                                               t_detection=t_puffs, 
                                               min_radius=min_radius_puffs, 
                                               ignore_frames=ignore_frames_loss
                                              )
        binary_waves = process_wave_prediction(pred=sample_waves,
                                               t_detection=t_waves, 
                                               min_radius=min_radius_waves, 
                                               ignore_frames=ignore_frames_loss
                                              )
        
        # combine all classes into a unique prediction
        binary_mask = 3*binary_puffs + 2*binary_waves + 1*binary_sparks
        assert np.all(binary_mask) < 4, "ROI in binary mask true for both puffs and waves"
        
        # write binary preds to disk
        load_epoch = epoch_2 if  training_name == epoch_2_training_name else epoch 
        filename = training_name + "_" + str(load_epoch) + "_" + video_id + "_pred_segmentation_raw_sparks.tif"
        imageio.volwrite(os.path.join(validation_folder, filename), np.uint8(binary_mask))